<a href="https://colab.research.google.com/github/atocf/chatTDAH/blob/main/chatTDAH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Imersão Inteligência Artificial 2ª Edição | Alura | Google
---



## Chatbot de Apoio a Estudantes com TDAH

**Objetivo:**
Desenvolver um chatbot interativo e personalizado destinado a estudantes com TDAH, visando facilitar seu gerenciamento de tempo, apoio na organização e suporte emocional. O chatbot fornecerá técnicas adaptadas, recursos educacionais e estratégias de bem-estar para ajudar os usuários a melhorar sua eficácia no estudo, a gestão de tarefas diárias e a saúde mental. Através de uma interface amigável e conteúdo multimídia, o chatbot busca criar um ambiente de aprendizagem inclusivo e suportável, onde os estudantes possam encontrar motivação, estratégias práticas de enfrentamento e orientação personalizada para superar os desafios impostos pelo TDAH no contexto acadêmico e pessoal.

**Link:**

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1nS0Duj78pRNRxQaAqelSuerikYK_AeJy?usp=sharing#scrollTo=HTiaTu6O1LRC"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/atocf/chatTDAH"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>
<br/><br/>



In [2]:
!pip install -q -U google-generativeai # Instala a biblioteca do Google Generative AI

In [3]:
!pip install --upgrade --force-reinstall gradio==3.50.2 # Instala a biblioteca Gradio para criar interfaces

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 5

In [1]:
# Importa a biblioteca pathlib para manipular caminhos de arquivo
import pathlib

# Importa a biblioteca textwrap para formatar texto
import textwrap

# Importa a biblioteca gradio para criar interfaces
import gradio as gr

# Importa o módulo os para interagir com o sistema operacional
import os

# Importa o módulo time para trabalhar com tempo
import time

# Importa a biblioteca Pandas para manipular dados
import pandas as pd

# Importa a biblioteca NumPy para operações numéricas
import numpy as np

# Importa a biblioteca Google Generative AI
import google.generativeai as genai

# Importa as funções display e Markdown do IPython para exibir elementos no Jupyter Notebook
from IPython.display import display, Markdown

# Define uma função para converter texto em Markdown, substituindo marcadores por '*' e indentado com '>'.
def to_markdown(text):
    text = text.replace('•', '  *')  # Substitui marcadores por '*'
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))  # Formata o texto com indentação

In [2]:
# Obtém a chave de API do Google Cloud
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key) # Configura a API com a chave obtida

In [8]:
# Configurações para geração de texto com o modelo Gemini
generation_config = {
  "temperature": 5, # Controla a aleatoriedade do texto gerado (0 = menos aleatório)
  "top_p": 1, # Controla a diversidade do texto gerado
  "top_k": 0, # Controla o número de palavras consideradas na geração
  "max_output_tokens": 2048  # Define o limite máximo de tokens na resposta
}

In [4]:
# Configurações de segurança para o modelo
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT", # Define a categoria de conteúdo prejudicial
    "threshold": "BLOCK_MEDIUM_AND_ABOVE" # Define o nível de sensibilidade para bloqueio
  },
  # Repete o padrão para outras categorias de conteúdo prejudicial
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

In [9]:
# Carrega o modelo Gemini Pro
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [6]:
# Cria um dataframe com as necessidades e opções de resposta
# Criação da DataFrame
df_necessidades = pd.DataFrame({
    "Necessidade": [
          "Apoio na Organização",
          "Ajustes de Expectativas",
          "Estratégias de Descanso e Pausa",
          "Estratégias de Gerenciamento de Tempo",
          "Feedback Positivo e Encorajamento",
          "Lembranças e Alertas",
          "Métodos de Estudo Adaptativos",
          "Recursos Visuais e Interativos",
          "Suporte Emocional e de Bem-Estar",
          "Técnicas de Manutenção do Foco"
    ],
    "Topico": [
         [
          	"Como organizar suas tarefas e prazos",
            "Dicas para manter a mesa de estudo organizada",
            "Estratégias para evitar a procrastinação",
            "Sistemas de organização para o TDAH",
            "Utilizando planners e aplicativos"
        ],
        [
          	"A importância de celebrar pequenas vitórias",
            "Definindo metas realistas",
            "Estratégias para ajustar objetivos ao longo do tempo",
            "Gerenciando expectativas pessoais e externas",
            "Lidando com a frustração e a autocrítica"
        ],
        [
          "A importância das pausas ativas",
          "Atividades de descompressão",
          "Exercícios rápidos de relaxamento",
          "Planejamento de intervalos no estudo",
          "Técnicas de mindfulness para estudantes"
        ],
        [
          "Como usar agendas e aplicativos de planejamento",
          "Dicas para evitar o sobrecarregamento",
          "Planejamento semanal para estudantes com TDAH",
          "Priorizando tarefas",
          "Técnicas de gerenciamento de tempo eficazes"
        ],
        [
          "A importância do autoelogio",
          "Construindo uma mentalidade positiva",
          "Criando uma rede de apoio",
          "Encontrando motivadores internos",
          "Técnicas para manter o encorajamento"
        ],
        [
          "Benefícios de ter uma rotina",
          "Configurando lembretes eficazes",
          "Dicas para não esquecer tarefas importantes",
          "Ferramentas digitais para alertas",
          "Personalizando alertas para suas necessidades"
        ],
        [
          "Como usar mapas mentais",
          "Dicas para leitura efetiva",
          "Estabelecendo um ambiente de estudo ideal",
          "Estratégias de memorização",
          "Técnicas de estudo para TDAH"
        ],
        [
          "Aplicativos úteis para TDAH",
          "Infográficos para organização de estudos",
          "Jogos e atividades interativas",
          "Plataformas de aprendizagem visual",
          "Vídeos educativos adaptados"
        ],
        [
          "Construindo resiliência",
          "Dicas para uma vida equilibrada",
          "Estratégias de coping para TDAH",
          "Gerenciando ansiedade e estresse",
          "Recursos de apoio e terapia"
        ],
        [
          "Dicas para manter a motivação",
          "Estratégias para melhorar a concentração",
          "Exercícios para treinar a atenção",
          "Minimizando distrações",
          "Uso de timers e aplicativos de foco"
        ]
    ]
})

In [10]:
# Texto explicativo para o usuário
texto_explicativo = """
**Olá! Bem-vindo ao seu assistente de apoio personalizado**

**Entendo que gerenciar o TDAH pode trazer desafios únicos, especialmente quando se trata de estudo, organização e bem-estar emocional. Estou aqui para oferecer recursos e estratégias adaptadas às suas necessidades. Quer você esteja procurando melhorar seu gerenciamento de tempo, encontrar métodos de estudo mais eficazes, ou simplesmente precisar de um pouco de encorajamento, estou aqui para ajudar**

Para começarmos, me diga um pouco sobre o que você está procurando hoje. Você pode me perguntar sobre:

  * **Estratégias de Gerenciamento de Tempo:** para planejar melhor seu dia e suas tarefas.
  * **Técnicas de Manutenção do Foco:** para minimizar distrações e melhorar a concentração.
  * **Apoio na Organização:** para dicas sobre como organizar seus estudos e seu espaço.
  * **Métodos de Estudo Adaptativos:** se você precisa de estratégias de estudo personalizadas.
  * **Lembranças e Alertas:** para configurar lembretes que te ajudem a manter o controle.
  * **Suporte Emocional e de Bem-Estar:** se estiver procurando por apoio emocional ou maneiras de gerenciar o estresse.
  * **Estratégias de Descanso e Pausa:** para aprender a fazer pausas eficazes que realmente recarregam suas energias.
  * **Recursos Visuais e Interativos:** se você aprende melhor com materiais visuais e interativos.
  * **Feedback Positivo e Encorajamento:** para construir uma mentalidade positiva.
  * **Ajustes de Expectativas:** se você está lutando para definir ou ajustar suas metas.

Digite o tópico de interesse ou me faça uma pergunta específica. Estou aqui para ajudar você a encontrar exatamente o que precisa!
"""

# Gera a mensagem inicial com a lista de necessidades
def gerar_mensagem_inicial():
    return "Olá! Como posso ajudá-lo hoje? Veja como posso ajudar:\n" + \
            "\n".join([f"{idx+1}. {nec}" for idx, nec in enumerate(df_necessidades["Necessidade"])])

estado_atual = {"fase": "necessidade", "necessidade_escolhida": None}

with gr.Blocks() as interface:
    gr.Markdown(texto_explicativo) # Exibe o texto explicativo
    chatbot = gr.Chatbot(
        bubble_full_width=False,
        value=[(None, gerar_mensagem_inicial())]  # Inclui a lista de necessidades na mensagem inicial
    )
    msg = gr.Textbox()

    def reiniciar_chat():
        # Reinicia o chat para o estado inicial
        estado_atual["fase"] = "necessidade"
        estado_atual["necessidade_escolhida"] = None
        return [("Bot", gerar_mensagem_inicial())]

    with gr.Row():
        None
        restart_button = gr.Button("Reiniciar Chat")

    # Botão para reiniciar o chat
    restart_button.click(fn=reiniciar_chat, inputs=[], outputs=chatbot)

    def respond(message, chat_history):
        # Processa a escolha do usuário
        try:
            escolha = int(message) - 1
            if estado_atual["fase"] == "necessidade":
                if escolha in range(len(df_necessidades)):
                    estado_atual["necessidade_escolhida"] = escolha
                    necessidade_escolhida = df_necessidades.iloc[escolha]["Necessidade"]
                    topicos = "\n".join([f"{idx+1}. {top}" for idx, top in enumerate(df_necessidades.iloc[escolha]["Topico"])])
                    bot_message = f"Aqui estão algumas opções de tópicos sobre {necessidade_escolhida}:\n{topicos}"
                    estado_atual["fase"] = "topico"
                    # Adiciona a escolha do usuário ao histórico antes da resposta do bot
                    chat_history.append((message, f"Você selecionou: {necessidade_escolhida}"))
                else:
                    bot_message = "Desculpe, não entendi. Por favor, escolha uma opção válida."
            elif estado_atual["fase"] == "topico":
                topicos = df_necessidades.iloc[estado_atual["necessidade_escolhida"]]["Topico"]
                if escolha in range(len(topicos)):
                    topico_escolhido = topicos[escolha]
                    necessidade_escolhida = df_necessidades.iloc[estado_atual["necessidade_escolhida"]]["Necessidade"]
                    prompt = f"Gostaria de saber mais sobre: {topico_escolhido}. Isso se relaciona com a minha dificuldade em {necessidade_escolhida}."
                    response = model.generate_content(prompt)
                    bot_message = response.text
                    # Reinicia para permitir nova escolha
                    estado_atual["fase"] = "necessidade"
                    estado_atual["necessidade_escolhida"] = None
                else:
                    bot_message = "Desculpe, não entendi. Por favor, escolha uma opção válida dentro dos tópicos."
        except ValueError:
            bot_message = "Por favor, insira um número correspondente à sua escolha."
        chat_history.append((None, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

if __name__ == "__main__":
    interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.2, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://7b4f216cfc3be5774f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
